In [2]:
import requests
from bs4 import BeautifulSoup
import math
import urlparse

payload = { 'method':'search', 'searchMethod':'true', 'searchTarget':'ATM',
 'orgName':'', 'orgId':'', 'hid_1':'1',
 'tenderName':'', 'tenderId':'', 'tenderStatus':'4,5,21,29',
 'tenderWay':'', 'awardAnnounceStartDate':'103/04/29',
 'awardAnnounceEndDate':'103/04/29', 'radProctrgCate':'3', 
 'proctrgCate':'3', 'tenderRange':'', 'minBudget':'',
 'maxBudget':'', 'item':'','hid_2':'1',
 'gottenVendorName':'', 'gottenVendorId':'', 'hid_3':'1',
 'submitVendorName':'', 'submitVendorId':'', 'location':'',
 'priorityCate':'', 'isReConstruct':'', 'btnQuery':'查詢' }

r = requests.session()
res = r.post('http://web.pcc.gov.tw/tps/pss/tender.do?searchMode=common&searchType=advance', data=payload)
response_text = res.text.encode('utf8')
soup = BeautifulSoup(response_text)
page_number  = int(math.ceil(float(soup.select('.T11b')[0].text) / 100)) 

bid_file = open("bid_list.txt", 'w')
page_format = "http://web.pcc.gov.tw/tps/pss/tender.do?searchMode=common&searchType=advance&searchTarget=ATM&method=search&isSpdt=&pageIndex=%d" 
for page in range(1, page_number + 1): 
    bid_list = r.get(page_format%(page)) 
    bid_response = bid_list.text.encode('utf8')
    soup = BeautifulSoup(bid_response)
    bid_table = soup.select("#print_area")[0]
    bid_rows = bid_table.select('tr')[1:-1] 
    for bid_row in bid_rows:
        #links = [tag['href'] for tag in bid_row.select('a')] 
        #print links[0] 
        #print "http://web.pcc.gov.tw/tps" + links[0][2:]    
        link = [tag['href'] for tag in bid_row.select('a')][0]    
        link_href = urlparse.urljoin("http://web.pcc.gov.tw/tps/pss/tender.do?searchMode=common&searchType=advance", link)   
        bid_file.write(link_href + "\n") 
bid_file.close()

In [4]:
# -*- coding: utf-8 -*- 
from bs4 import BeautifulSoup 
import re, requests 

bid_list = open('bid_list.txt', 'r') 
for line in bid_list.readlines(): 
    pagelink = line.strip() 
    m = re.match(r"([^ ]+)pkAtmMain=(?P<pkAtmMain>.+)&tenderCaseNo=(?P<tenderCaseNo>.+)", pagelink) 
    filename = "%s_%s"%(m.group('pkAtmMain'), m.group('tenderCaseNo')) 
    
    request_get = requests.get(pagelink) 
    response = request_get.text.encode('utf8') 
    soup = BeautifulSoup(response) 
    printarea = soup.select('#printArea')[0] 
    bid_detail = open("gov/%s.txt"%(filename), 'w') 
    bid_detail.write(printarea.prettify("utf-8")) 
    bid_detail.close() 
bid_list.close()